In [8]:
import pandas as pd, matplotlib.pyplot as plt, matplotlib.patches as patches, re, calendar
from collections import defaultdict
from matplotlib.dates import DayLocator, DateFormatter
from pathlib import Path

# =========================
# Settings
# =========================
EXCEL_PATH = "SM_courses_information.xlsx"
OUT_DIR = Path("gantt_out")
MONTHS_TO_PLOT = [(2025, 9), (2025, 10), (2025, 11), (2025, 12), (2026, 1)]

# base palette (can be any length; will be expanded/cycled to match number of types)
PALETTE = [
     "#66CEFA",  # deep blue
    "#6285C7",  # turquoise
    "#5BB7C5",  # light aqua
    "#A3D0FF",  # pastel blue
]

COLORS = {
    "online": "#646464",
    "inperson": "#003caa",
    "assignment": "#7B0450",
    "exam": "#EF5391",
}

# TYPE_COLORS will be built dynamically in main() based on Course_Type values found in the file
TYPE_COLORS = {}

DATE_RE = re.compile(r"(\d{2}\.\d{2}\.\d{4}|\d{4}-\d{2}-\d{2})")

# =========================
# Helpers
# =========================
def _extract_dates(cell):
    if pd.isna(cell):
        return []
    s = str(cell).strip()
    if s in {"", "-", "—", "–", "None", "nan", "NaT"}:
        return []
    tokens = DATE_RE.findall(s) or [t.strip() for t in s.split(",")]
    out = set()
    for t in tokens:
        try:
            out.add(pd.to_datetime(t, dayfirst=("." in t)).normalize())
        except Exception:
            pass
    return sorted(out)


def _read_excel_records(path: str):
    """Read excel and aggregate dates per course, also capture Course_Type (ffill)."""
    xls = pd.ExcelFile(path)
    sheet = xls.sheet_names[0]
    df = pd.read_excel(xls, sheet_name=sheet)
    df.columns = [str(c).strip() for c in df.columns]

    # forward-fill course names (if a course spans multiple rows)
    if "Course_Name" in df.columns:
        df["Course_Name"] = df["Course_Name"].ffill()

    # forward-fill course types if column exists
    if "Course_Type" in df.columns:
        df["Course_Type"] = df["Course_Type"].ffill()
    else:
        # ensure column exists (all None) so getattr won't fail
        df["Course_Type"] = None

    # ensure these columns exist to avoid attribute errors
    for col in ["Online_Classes_Dates", "InPerson_Classes_Dates", "Assignments_Dates", "EXAM"]:
        if col not in df.columns:
            df[col] = None

    recmap = defaultdict(lambda: {
        "Course": "",
        "Type": "Other",
        "OnlineDates": set(),
        "InPersonDates": set(),
        "Assignments": set(),
        "Exams": set()
    })

    for r in df.itertuples(index=False):
        cname = str(getattr(r, "Course_Name", "")).strip()
        if not cname:
            continue
        rec = recmap[cname]
        rec["Course"] = cname

        # capture / normalize course type (use "Other" if empty)
        ctype = getattr(r, "Course_Type", None)
        if ctype is None or str(ctype).strip() in {"", "nan", "None"}:
            # keep existing rec["Type"] if already set by previous row (ffill)
            pass
        else:
            rec["Type"] = str(ctype).strip()

        rec["OnlineDates"].update(_extract_dates(getattr(r, "Online_Classes_Dates", None)))
        rec["InPersonDates"].update(_extract_dates(getattr(r, "InPerson_Classes_Dates", None)))
        rec["Assignments"].update(_extract_dates(getattr(r, "Assignments_Dates", None)))
        rec["Exams"].update(_extract_dates(getattr(r, "EXAM", None)))

    records = []
    for course, d in recmap.items():
        if d["OnlineDates"] or d["InPersonDates"] or d["Assignments"] or d["Exams"]:
            records.append({
                "Course": course,
                "Type": d.get("Type", "Other"),
                "OnlineDates": sorted(d["OnlineDates"]),
                "InPersonDates": sorted(d["InPersonDates"]),
                "Assignments": sorted(d["Assignments"]),
                "Exams": sorted(d["Exams"]),
            })
    return records


def _month_bounds(year, month):
    first = pd.Timestamp(f"{year}-{month:02d}-01")
    last = pd.Timestamp(f"{year}-{month:02d}-{calendar.monthrange(year, month)[1]}")
    return first, last


def _date_formatter():
    try:
        return DateFormatter("%-d.%-m.")
    except Exception:
        return DateFormatter("%d.%m.")


# =========================
# Graph
# =========================
def plot_month(records, year, month, out_file, type_colors):
    start, end = _month_bounds(year, month)
    fig, ax = plt.subplots(figsize=(20, 9))
    bar_h = 0.7
    yticks, labels = [], []

    for rec in sorted(records, key=lambda x: x["Course"]):
        markers = [d for d in rec["OnlineDates"] + rec["InPersonDates"] + rec["Assignments"] + rec["Exams"] if start <= d <= end]
        if not markers:
            continue
        y = len(yticks)
        yticks.append(y)
        labels.append(rec["Course"])

        # choose color by course type (fallback to "Other")
        ctype = rec.get("Type", "Other") or "Other"
        color = type_colors.get(ctype, type_colors.get("Other", PALETTE[0]))

        block_start, block_end = min(markers), max(markers)
        ax.add_patch(patches.Rectangle((block_start, y - bar_h/2), (block_end - block_start), bar_h,
                                       linewidth=0, facecolor=color, alpha=0.6))

        for x in rec["OnlineDates"]:
            if start <= x <= end:
                ax.vlines(x, y - bar_h/2, y + bar_h/2, colors=COLORS["online"], linewidth=3)

        for x in rec["Exams"]:
            if start <= x <= end:
                ax.vlines(x, y - bar_h/2, y + bar_h/2, colors=COLORS["exam"], linewidth=4)
                ax.plot(x, y, marker="o", markersize=10, color=COLORS["exam"], markeredgecolor="black", markeredgewidth=0.6)

        for x in rec["Assignments"]:
            if start <= x <= end:
                ax.vlines(x, y - bar_h/2, y + bar_h/2, colors=COLORS["assignment"], linewidth=4)
                ax.plot(x, y, marker="^", markersize=12, color=COLORS["assignment"], markeredgecolor="black", markeredgewidth=0.6)

        for x in rec["InPersonDates"]:
            if start <= x <= end:
                ax.vlines(x, y - bar_h/2, y + bar_h/2, colors=COLORS["inperson"], linewidth=4)
                ax.plot(x, y, marker="o", markersize=10, color=COLORS["inperson"], markeredgecolor="black", markeredgewidth=0.6)

    # --- Legends
    # course type legend (color patches)
    type_legend = [patches.Patch(facecolor=col, label=typ, alpha=0.8) for typ, col in type_colors.items()]
    legend1 = ax.legend(handles=type_legend, loc="upper right", frameon=True, title="Course types")
    ax.add_artist(legend1)

    # milestone legend (icons)
    from matplotlib.lines import Line2D
    milestone_legend = [
        Line2D([0], [0], color=COLORS["online"], lw=3, label="Online class"),
        Line2D([0], [0], color=COLORS["inperson"], lw=4, marker='o', markersize=8,
               markerfacecolor=COLORS["inperson"], markeredgecolor="black", label="In-person class"),
        Line2D([0], [0], color=COLORS["assignment"], lw=4, marker='^', markersize=10,
               markerfacecolor=COLORS["assignment"], markeredgecolor="black", label="Assignment"),
        Line2D([0], [0], color=COLORS["exam"], lw=4, marker='o', markersize=9,
               markerfacecolor=COLORS["exam"], markeredgecolor="black", label="Exam"),
    ]
    ax.legend(handles=milestone_legend, loc="upper center", frameon=True, title="Milestones")

    ax.set_xlim(start, end)
    ax.set_ylim(-1, max(1, len(yticks)))
    ax.set_yticks(range(len(yticks)))
    ax.set_yticklabels(labels)
    ax.set_title(f"Gantt — {start.strftime('%B %Y')}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Course")
    ax.xaxis.set_major_locator(DayLocator(interval=1))
    ax.xaxis.set_major_formatter(_date_formatter())
    ax.tick_params(axis='x', labelrotation=45)
    ax.grid(True, axis="x", linestyle="--", alpha=0.35)
    plt.tight_layout()
    out_file.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(out_file, dpi=200)
    plt.close(fig)


# =========================
# Main
# =========================
def build_type_colors(records, base_palette):
    """Create TYPE_COLORS mapping from unique types found in records.
       Palette will be extended (cycled) if not long enough."""
    types = sorted({(r.get("Type") or "Other").strip() for r in records})
    # normalize empty-like values
    types = [t if t not in {"", "nan", "None"} else "Other" for t in types]
    if not types:
        return {"Other": base_palette[0]}
    # ensure palette length == number of types: if too short, cycle; if longer, trim
    needed = len(types)
    # extend palette by cycling
    extended = [base_palette[i % len(base_palette)] for i in range(needed)]
    type_colors = {typ: extended[i] for i, typ in enumerate(types)}
    return type_colors


def main():
    records = _read_excel_records(EXCEL_PATH)

    # build TYPE_COLORS dynamically based on Course_Type values present in the file
    global TYPE_COLORS
    TYPE_COLORS = build_type_colors(records, PALETTE)

    print("Course types found and colors assigned:")
    for t, c in TYPE_COLORS.items():
        print(f"  {t}: {c}")

    for y, m in MONTHS_TO_PLOT:
        plot_month(records, y, m, OUT_DIR / f"gantt_{y}-{m:02d}.png", TYPE_COLORS)
    print(f"✅ Done: {OUT_DIR.resolve()}")


if __name__ == "__main__":
    main()


Course types found and colors assigned:
  ILV: #66CEFA
  ILV - 1: #6285C7
  SE: #5BB7C5
  VO - 1: #A3D0FF
✅ Done: /Users/baskaklimek/Documents/gantt/gantt_out
